# Analysis of a Dice Tower

In which I investigate the randomness properties of dice rolled using a home-made dice tower, with the techniques given by Donald Knuth in *The Art of Computer Programming, Vol. 2: Seminumerical Algorithms.*

In [1]:
import math
import scipy.stats
import matplotlib.pyplot as plt

# Number formatting fuctions
def f(number): return f'{round(number, 2)}'
def p(number): return f'{round(number * 100, 1)}%'

# Factorial!
def fact(n):
    'n!'
    if n <= 1: return 1
    else:      return n * fact(n - 1)

In [2]:
def count(sample, n):
    'Return counts of each element of sample, '
    'where each sample is 0..n'
    count = [0] * n
    for elt in sample:
        count[elt - 1] += 1
    return count

The Stirling numbers ${n \brace m}$ can be computed with the function

$$
S(n,m) =
\begin{cases}
               1 & \textrm{if } m = 1 \\
               1 & \textrm{if } n = m \\
               0 & \textrm{if } n < m \\
               m S(n-1,m) + S(n-1,m-1) & \textrm{otherwise}
\end{cases}
$$

See https://people.sc.fsu.edu/~jburkardt/py_src/polpak/stirling2.py.

In [3]:
def Stirling(n, m):
    if   m == 1: return 1
    elif m == n: return 1
    elif n < m:  return 0
    else:
        return (
            (m * Stirling(n - 1, m))
            + Stirling(n - 1, m - 1)
        )

In [4]:
def by_n(sequence, n):
    'iterate over n-element slices of sequence'
    for i in range(0, len(sequence), n):
        yield list(sequence[i:i+n])

def by_n_overlap(sequence, n):
    'iterate over every n-element slice of sequence'
    for i in range(0, len(sequence) + 1 - n):
        yield list(sequence[i:i+n])

## Methodology

The dice tower in question was assembled from a USPS priority mail box (used) and blue painter's tape in about 10 minutes, without the use of any form of plans. It has three baffles with an additional ramp to convice the dice to exit the tower, since the baffles are in the wrong positions.

The dice are three pink d6, about 10mm, maybe. They had the most rounded edges of my dice collection and were not very large

### Dice rolling

The dice were rolled one at a time by holding the 1 pip up approximately 25mm above the dice tower top opening and releasing them under gravity. Three dice were rolled in each trial---the value chosen is the value initially rolled on the dice, which may be changed by the impact of a later dice.

`rolls` is the result of rolling dice.

`repetition` is a sequence of repetitions of 1-6, with a length equal to the length of rolls, as one control test.

`sorted_rolls` is `rolls`, sorted, as another control test. (Obviously, it would be locally very non-random.)

`duplicated` is `rolls`, with each element duplicated, like [3, 3, 1, 1, ...], as another control test.

`normal` is a sequence of rounded values from a truncated normal distribution over [1,6], as a random, but non-uniformly distributed control test.

`uniform` is a sequence of values from scipy.stats' uniform distribution, which are $0 \leq x \lt 1$ and then moved into the range [1,6] by $\lfloor{6x}\rfloor + 1$. This is another control test which should be positive.

In [5]:
rolls = list(map(lambda n: int(n),
    '313141462543662425625132222521641644463233423221345122666465'
    '366423246553145466524122645362513543543326655236561436422616'
    '611613112624535652352564164532146152441526435512423115163413'
    '332151643426566431425635221141124661516423215513532534554326'
    '522343666132235566231264113132446115521541113216412524426615'
    '342553344426541534655246136631251362246243634634433312125265'
    '224612651221552336624611164243633621245115452152444416362232'
    '156456653234361315626124622261421354415451442424356412121533'
    '225316344434454125162115662514312255654355331332432154143522'
    '634454125523651352225262335662232312136343135163465454621524'
    '564152663235435612156423544333544613412344463522624634261616'
    '615566414143361664411553343521515552363331315216235254625326'
    '434346612531512565511563446423646416365322431663354221225145'
    '543654126651535163436662341165266516511552615233141216621142'
    '513442634663325356235441352552432664231266451126344635165441'
    '426555345332551566551464125335125465266111433513342644251636'
    '343153241542132252165135336416443466334'
))

repetition = [1,2,3,4,5,6] * (len(rolls) // 6)

sorted_rolls = list(rolls)
sorted_rolls.sort()

duplicated = [
    item
    for sublist in map(lambda x: [x, x], rolls)
    for item in sublist
]

normal = scipy.stats.truncnorm(a=1/3, b=6/3, scale=3).rvs(len(rolls)).round().astype(int)

uniform = list(map(lambda x: math.floor(6*x) + 1, scipy.stats.uniform.rvs(size=len(rolls))))

len(uniform)

999

## Tests

### Chi-square

The $\chi^2$ test is a good ol' staple of statistical statistics. It compares an experimental random sequence, `sample`, with an expected random distribution, `f_exp`, producing a value, `statistic`, or a `pvalue`, the probability of getting a result as "extreme" as the sample from the expected random distribution. By default, the expected distribution is the uniform random distribution. The $\chi^2$ test is useful when each sample value comes from a bin, like a 6-sided dice roll that produces 1, 2, 3, 4, 5, or 6.

In [6]:
def chi2test(sample, pvalue=True, f_exp=None):
    'Perform the chi-squared test on counts, '
    'normally returning the pvalue but optionally '
    'the statistic value.'
    result = scipy.stats.chisquare(
        sample,
        f_exp = f_exp
    )
    if pvalue:
        return result.pvalue
    else:
        return result.statistic

Applying the $\chi^2$ test to the test sequences above gives:

| Sequence | $\chi^2$ |
|:--------:|:-----------------:|
| `rolls` | {{ p(chi2test(count(rolls, 6))) }} |
| `repetition` | {{ p(chi2test(count(repetition, 6))) }} |
| `sorted_rolls` | {{ p(chi2test(count(sorted_rolls, 6))) }} |
| `duplicated` | {{ p(chi2test(count(duplicated, 6))) }} |
| `normal` | {{ p(chi2test(count(normal, 6))) }} |
| `uniform` | {{ p(chi2test(count(uniform, 6))) }} |

**The $\chi^2$ *pvalue* for the `rolls` is {{ p(chi2test(count(rolls, 6))) }}** (representing the chance that the result of a true uniform distribution would be at least as extreme as this sample), which is reasonable. (Basically, anything that isn't very close to 0% or 100% is "reasonable". Random numbers are ... random.)

**The $\chi^2$ value for `sorted_rolls` is also {{ p(chi2test(count(sorted_rolls, 6))) }},** and **the $\chi^2$ value for `duplication` is {{ p(chi2test(count(duplicated, 6))) }},** both also acceptable.

**The $\chi^2$ value for the `repetition` is {{ p(chi2test(count(repetition, 6))) }} and that for `normal` is {{ p(chi2test(count(normal, 6))) }}.** Those don't seem reasonable.

Finally, **the $\chi^2$ value for `uniform` is {{ p(chi2test(count(uniform, 6))) }},** which is acceptable.

Knuth suggests making tests on smaller sub-sequences of the overall random sequence being tested, in order to look at local behavior.

In [7]:
def sub_chi2test(samples, n, pvalue=True):
    '''Compute χ^2 values for n-element subsequences of lst'''
    return [
        chi2test(count(sublst, 6), pvalue=pvalue)
        for sublst in by_n(samples, n)
    ]

**The $\chi^2$ *pvalue* of each 60-element subsequences of `rolls` is {{ ', '.join(map(p, sub_chi2test(rolls, 60))) }};** most of the values are acceptable although some are a little sketchy. **The same values for `repetition` is {{ ', '.join(map(p, sub_chi2test(repetition, 60))) }},** which is as bad as the value of the whole sequence. On the other hand, if we break it at 61, the subsequences are {{ ', '.join(map(p, sub_chi2test(repetition, 61))) }}, which is bad in a different way.

**The $\chi^2$ *pvalue* of `sorted_rolls` is {{ p(chi2test(count(sorted_rolls, 6))) }}, the same as the unsorted list, but the subsequence test is {{ ', '.join(map(p, sub_chi2test(sorted_rolls, 60))) }}, clearly indicating something is wrong.**

**The subsequence test for `normal` is {{ ', '.join(map(p, sub_chi2test(normal, 60))) }},** which seems suspicious.

### Kolmogorov-Smirnov

(There should be vodka for this.)

In [8]:
def kstest(samples, cdf=scipy.stats.uniform.cdf):
    'Compute KS test for samples, compared to distribution cdf'
    return scipy.stats.kstest(samples, cdf).pvalue

The Kolmogorov-Smirnov test is fundamentally similar to the $\chi^2$ test. The KS *pvalue* for the rolls is {{ p(kstest(count(rolls, 6))) }} (representing the chance that the result of a true uniform distribution would be less than this sample), which is not good. The problem, as Knuth notes, is

> The Kolmogorov-Smirnov test (KS test) may be used when $F(x)$ has no jumps.

KS is useful for continuous distributions, such as what are typically produced by computer pseudo-random number generators---typically floating point numbers between 0 and 1---while the $\chi^2$ test is useful for discrete distributions such as this case where the result is an integer between 1 and 6  meaning that they are entirely jumps. In the case of dice, the $\chi^2$ test is more appropriate.

There is, however, one good use of Kolmogorov-Smirnov with dice rolling: if we break up the rolls into subsequences, perform the $\chi^2$ test on the subsequences, and then use the KS test to combine the results, the end result is a single value for the localized behavior of the sequence. This *is* a reasonable thing to do, because the statistic result of the $\chi^2$ test comes from the $\chi^2$ distribution, which is continuous.

In order to combine the $\chi^2$ results in this way, we need to get the $\chi^2$ statistics and compare their distributions to the $\chi^2$ cumulative distribution function with the appropriate degrees of freedom (5 in the case of 6-sided dice rolls). (And no, you are not expected to understand that last remark. I certainly don't.)

In [9]:
def ks_combine_chi2(samples, n):
    'Compute χ^2 values for n-element subsequences of lst, '
    'then combine them with the KS test using χ^2 distribution'
    df = max(samples) - 1
    return kstest(
        sub_chi2test(samples, n, pvalue=False),
        cdf=lambda x: scipy.stats.chi2.cdf(x, df)
    )

| Sequence | Combined $\chi^2$ |
|:--------:|:-----------------:|
| `rolls` | {{ p(ks_combine_chi2(rolls, 30)) }} |
| `repetition` | {{ p(ks_combine_chi2(repetition, 30)) }} |
| `sorted_rolls` | {{ p(ks_combine_chi2(sorted_rolls, 30)) }} |
| `duplicated` | {{ p(ks_combine_chi2(duplicated, 30)) }} |
| `normal` | {{ p(ks_combine_chi2(normal, 30)) }} |
| `uniform` | {{ p(ks_combine_chi2(uniform, 30)) }} |

**Combining the results of the 30-element $\chi^2$ subsequence test on `rolls` produces {{ p(ks_combine_chi2(rolls, 30)) }} and the `uniform` sample produces {{ p(ks_combine_chi2(uniform, 30)) }}, while the `repetition` sample produces {{ p(ks_combine_chi2(repetition, 30)) }}, the `sorted_rolls` sample produces {{ p(ks_combine_chi2(sorted_rolls, 30)) }}, the `duplicated` sample is {{ p(ks_combine_chi2(duplicated, 30)) }}, and the `normal` sample produces {{ p(ks_combine_chi2(normal, 30)) }}.** The first two are good, the rest are not.

### Empirical tests

Knuth presents 10 "empirical" tests "applied to sequences in order to investigate their randomness".

For an example, the first of these tests is the *Equidistribution test* or *Frequency test*, that the numbers from the sequence are uniformly distributed across their range. One alternative is to plot a histogram.

**Histogram for `rolls`**

{{ _ = plt.hist(x=rolls, bins=6, color='#0504aa', rwidth=0.85) }}

**Histogram for `repetition`**

{{ _ = plt.hist(x=repetition, bins=6, color='#0504aa', rwidth=0.85) }}

At least it's lovely and even.

**Histogram for `sorted_rolls`**

{{ _ = plt.hist(x=sorted_rolls, bins=6, color='#0504aa', rwidth=0.85) }}

Weirdly similar to `rolls`.

**Histogram for `duplicated`**

{{ _ = plt.hist(x=duplicated, bins=6, color='#0504aa', rwidth=0.85) }}

A slightly more emphatic version of `rolls`.

**Histogram for `normal`**

{{ _ = plt.hist(x=normal, bins=6, color='#0504aa', rwidth=0.85) }}

Very definitely odd.

**Histogram for `uniform`**

{{ _ = plt.hist(x=uniform, bins=6, color='#0504aa', rwidth=0.85) }}

But Knuth points out that determining whether the sequence elements are uniformly distributed across their range is one of the purposes of the $\chi^2$ and KS tests. So that seems rather taken care of.

#### Serial test

This test determines whether sub-sequences of successive numbers are uniformly distributed in an independent matter. It does this by counting how often given sub-sequences (i.e. $s_i, ..., s_{i+n}$ for a sub-sequence of length $n+1$) and then running the $\chi^2$ test against the counts.

In [10]:
def inc_bin(ary, n, loc):
    elt = 0
    for i in loc:
        elt = (elt * n) + (i - 1)
    ary[elt] += 1

def serial_test(sample, n):
    ary = [0] * (6 ** n)
    for loc in by_n_overlap(sample, n):
        inc_bin(ary, 6, loc)
    return chi2test(ary)

The serial test values for the various sequences are:

| Sequence | Length 2 | Length 3 |
|:--------:|:--------:|:--------:|
| `rolls`  | {{ p(serial_test(rolls, 2)) }} | {{ p(serial_test(rolls, 3)) }} |
| `repetition`  | {{ p(serial_test(repetition, 2)) }} | {{ p(serial_test(repetition, 3)) }} |
| `sorted_rolls`  | {{ p(serial_test(sorted_rolls, 2)) }} | {{ p(serial_test(sorted_rolls, 3)) }} |
| `duplicated`  | {{ p(serial_test(duplicated, 2)) }} | {{ p(serial_test(duplicated, 3)) }} |
| `normal` | {{ p(serial_test(normal, 2)) }} | {{ p(serial_test(normal, 3)) }} |
| `uniform` | {{ p(serial_test(uniform, 2)) }} | {{ p(serial_test(uniform, 3)) }} |

**The serial test does a very good job of separating `rolls` and `uniform` from samples with non-uniformly random local behavior.** Unfortunately, serial lengths much above 3 lead to large numbers of bins for the $\chi^2$ and reduced counts in the bins.

#### Gap test

The gap test examines the length of "gaps" between two occurrences of a given value. I'm totally ignoring Knuth's algorithm for this because it is intended for continuous values and because I just don't see what he's getting at for them.

The probability distribution that the gap lengths should be compared against is:

$$
p_r = 
\begin{cases}
p (1 - p)^r & \textrm{if } 0 \leq r \lt t \\
(1 - p)^r     & \textrm{if } r = t
\end{cases}
$$

for $0 \leq r \leq t$ and where $p$ is the probability of a dice rolling a given side, $1/\textrm{bins}$. When $0 \leq r \lt t$, $r$ the index into the counts and $t$ is a catch-all gap length to limit the number of counts---any gap longer than $t$ is counted in $t$.

In [11]:
def count_gaps(sample, bins):
    'Count the gaps between subsequent occurances '
    'of each element of sample. The elements of the '
    'sample should be in 0..bins.'
    counts = {}
    seen = {}
    for i in range(0, bins): seen[i] = 0
    for elt in sample:
        for i in range(0, bins):
            if i == (elt - 1):
                # update counts for current elt
                if seen[i] in counts:
                    counts[seen[i]] += 1
                else:
                    counts[seen[i]] = 1
                seen[i] = 0
            else:
                # increment seen for non-current elt
                seen[i] += 1
    # finish counts at end of sample
    for i in seen:
        if seen[i] in counts:
            counts[seen[i]] += 1
        else:
            counts[seen[i]] = 1
    return [
        counts[k] if k in counts else 0
        for k in range(0, max(counts) + 1)
    ]

def gap_test(sample, bins):
    'Perform the gap test on sample, returning a '
    'pvalue. The elements of the sample should be '
    'in 0..bins.'
    counts = count_gaps(sample, bins)
    p = 1 / bins
    n = len(counts)
    s = sum(counts)
    f_exp = [
        s * p * pow(1 - p, i)
        for i in range(0, n)
    ]
    f_exp[n - 1] = s * pow(1 - p, n - 1)
    return chi2test(
        counts,
        f_exp = f_exp
    )

The gap test values for the various samples are:

| Sequence | Gap test |
|:--------:|:--------:|
| `rolls`  | {{ p(gap_test(rolls, 6)) }} |
| `repetition`  | {{ p(gap_test(repetition, 6)) }} |
| `sorted_rolls`  | {{ p(gap_test(sorted_rolls, 6)) }} |
| `duplicated`  | {{ p(gap_test(duplicated, 6)) }} |
| `normal` | {{ p(gap_test(normal, 6)) }} |
| `uniform` | {{ p(gap_test(uniform, 6)) }} |

Once again, **the gap test does a good job of differentiating `rolls` from the other samples,** whose gaps do not match the expected distribution for various reasons.

#### Poker test

The probability distribution that the poker test should be compared against is:

$$
p_r = \frac{\Pi_{0 \leq i \leq r + 1}(d - i)}{d^k}{k \brace r}
$$

Where $d$ is the number of bins, $0 \leq r \lt \textrm{bins}$ and $k$ is the size of the poker hand.

In [12]:
def poker_dist(d, k, r):
    prod = 1
    for t in range(d, d - r, -1):
        prod *= t
    return Stirling(k, r) * prod / pow(d, k)

def poker_test(sample, n, bins=6):
    sequence = by_n_overlap(sample, n)
    counts = count(
        map(lambda elt: len(set(elt)), sequence),
        n
    )
    return chi2test(
        counts,
        f_exp = [
            sum(counts) * poker_dist(bins, n, r)
            for r in range(1, bins)
        ]
    )

| Sequence | Poker test |
|:--------:|:--------:|
| `rolls`  | {{ p(poker_test(rolls, 5)) }} |
| `repetition`  | {{ p(poker_test(repetition, 5)) }} |
| `sorted_rolls`  | {{ p(poker_test(sorted_rolls, 5)) }} |
| `duplicated`  | {{ p(poker_test(duplicated, 5)) }} |
| `normal` | {{ p(poker_test(normal, 5)) }} |
| `uniform` | {{ p(poker_test(uniform, 5)) }} |


#### Coupon collector's test

$$
p_r =
\begin{cases}
\frac{d!}{d^r}{r - 1 \brace d - 1} & \textrm{if } d \leq r \lt t \\
1 - \frac{d!}{d^{t-1}}{t - 1 \brace d} & \textrm{if } r = t
\end{cases}
$$

In [13]:
def coupon_count(sample, bins):
    count  = {}
    occurs = [False] * bins
    length = 0
    for x in sample:
        occurs[x - 1] = True
        length += 1
        if all(occurs):
            if length in count:
                count[length] += 1
            else:
                count[length] = 1
            length = 0
            occurs = [False] * bins
    # build result array
    # min len(result) = 2 * bins
    # skip the first bins elements
    # because they're always 0
    counts = max(count) + 1 - bins
    min_counts = 2 * bins
    result = [
        count[k] if k in count else 0
        for k in range(
            bins,
            bins + max(counts, min_counts)
        )
    ]
    # maximum len(result) = 5 * bins
    max_counts = 5 * bins
    if len(result) > max_counts:
        s = sum(result[max_counts:])
        result = result[:max_counts]
        result[max_counts - 1] += s
    return result
    
def coupon_test(sample, bins):
    count  = coupon_count(sample, bins)
    total  = sum(count)
    counts = len(count)
    f_exp = [
        total * ((fact(bins) / pow(bins, r)) * Stirling(r - 1, bins - 1))
        for r in range(bins, counts + bins - 1)
    ]
    f_exp.append(
        total * (1 - ((fact(bins) / pow(bins, counts - 1)) * Stirling(counts - 1, bins)))
    )
    return chi2test(count, f_exp=f_exp)

| Sequence | Coupon collector's test |
|:--------:|:--------:|
| `rolls`  | {{ p(coupon_test(rolls, 6)) }} |
| `repetition`  | {{ p(coupon_test(repetition, 6)) }} |
| `sorted_rolls`  | {{ p(coupon_test(sorted_rolls, 6)) }} |
| `duplicated`  | {{ p(coupon_test(duplicated, 6)) }} |
| `normal` | {{ p(coupon_test(normal, 6)) }} |
| `uniform` | {{ p(coupon_test(uniform, 6)) }} |


#### Permutation test

In [14]:
def max_loc(perm, end):
    loc = 0
    max = perm[0]
    for i in range(1, end):
        if perm[i] > max:
            loc = i
            max = perm[i]
    return loc

def perm_int(perm):
    f = 0
    for r in range(len(perm), 1, -1):
        s = max_loc(perm, r) + 1
        f = r * f + s - 1
        perm[r - 1], perm[s - 1] = perm[s - 1], perm[r - 1]
    return f

def permutation_count(sample, n):
    count = [0] * fact(n)
    for seq in by_n(sample, n):
        pi = perm_int(seq)
        count[pi] += 1
    return count

def permutation_test(sample, n):
    return chi2test(permutation_count(sample,n))

(
    permutation_test(rolls, 3), permutation_test(uniform, 3), 
    permutation_test(list(scipy.stats.norm.rvs(size=len(rolls))), 3)
)

(4.739064889233372e-06, 1.9127050172830083e-06, 0.7305096429627241)


| Sequence | Permutation test |
|:--------:|:--------:|
| `rolls`  | {{ p(permutation_test(rolls, 3)) }} |
| `repetition`  | {{ p(permutation_test(repetition, 3)) }} |
| `sorted_rolls`  | {{ p(permutation_test(sorted_rolls, 3)) }} |
| `duplicated`  | {{ p(permutation_test(duplicated, 3)) }} |
| `normal` | {{ p(permutation_test(normal, 3)) }} |
| `uniform` | {{ p(permutation_test(uniform, 3)) }} |

**`rolls` histogram**

{{ _ = plt.bar(x=[0,1,2,3,4,5], height=permutation_count(rolls, 3), color='#0504aa', width=0.85) }}

**`uniform` histogram**

{{ _ = plt.bar(x=[0,1,2,3,4,5], height=permutation_count(uniform, 3), color='#0504aa', width=0.85) }}


#### Run test

In [15]:
def runs_up(sample, n):
    count = [0] * n
    k = 1
    i = 0
    while i < len(sample) - 1:
        if sample[i] <= sample[i+1]:
            k += 1
            i += 1
        else:
            if k < n:
                count[k - 1] += 1
            else:
                count[n - 1] += 1
            k = 1
            i += 2
    if k < n:
        count[k - 1] += 1
    else:
        count[n - 1] += 1
    return count

def runs_fexp(s, n):
    return [
        s * ((1 / fact(r)) - (1 / fact(r + 1)))
        for r in range(1, n + 1)
    ]

def runs_test(sample, n):
    c = runs_up(sample, n)
    s = sum(c)
    f = runs_fexp(s, n)
    return chi2test(c, f_exp=f)

def runs_bar(sample, n):
    width = 0.45
    xs = range(0,n)
    count = runs_up(rolls, n)
    plt.bar(x=xs, height=count, color='#0504aa', width=-0.4, align='edge', label='Sample')
    plt.bar(x=xs, height=runs_fexp(sum(count), n), color='#aa0405', width=0.4, align='edge', label='Expected')
    plt.legend()

(
runs_test(list(scipy.stats.uniform.rvs(size=len(rolls))), 5),
runs_test(list(scipy.stats.norm.rvs(size=len(rolls))), 5)
)


(0.7353814964020309, 0.32035393791619565)


| Sequence | Permutation test |
|:--------:|:--------:|
| `rolls`  | {{ p(runs_test(rolls, 5)) }} |
| `repetition`  | {{ p(runs_test(repetition, 5)) }} |
| `sorted_rolls`  | {{ p(runs_test(sorted_rolls, 5)) }} |
| `duplicated`  | {{ p(runs_test(duplicated, 5)) }} |
| `normal` | {{ p(runs_test(normal, 5)) }} |
| `uniform` | {{ p(runs_test(uniform, 5)) }} |

**`rolls` histogram**

{{ runs_bar(rolls, 5) }}

**`uniform` histogram**

{{ runs_bar(uniform, 5) }}


#### Maximum-of-t test

In [16]:
def max_of_t_test(sample, n):
    s = [ max(seq) for seq in by_n(sample, n)]
    c = count(s, 6)
    f = [ pow(y, n) for y in range(1,7)]
    return chi2test(c, f_exp=f)

max_of_t_test(rolls, 5)

0.0

#### Collision test

#### Serial correlation test

In [17]:
def corr_rng(n):
    mu = -1 / (n - 1)
    sig = (1 / (n - 1)) * math.sqrt((n * (n - 3)) / (n + 1))
    return (mu - (2 * sig), mu + (2 * sig))

def correlation_coef(sample, n):
    corr = scipy.stats.pearsonr(
        sample,
        [sample[(i+n) % len(sample)] for i in range(0, len(sample))]
    )[0]
    rng = corr_rng(len(sample))
    return (corr, rng, rng[0] < corr and corr < rng[1])

correlation_coef(rolls, 1)

def cor(sample, n):
    cr = correlation_coef(sample, n)
    return f'{cr[2] and "Valid" or "Invalid"} ({f(cr[0])})'


| Sequence | 1 | 2 | 4 | 6 |
|:--------:|:-:|:-:|:-:|:-:|
| `rolls`  | {{ cor(rolls, 1) }} | {{ cor(rolls, 2) }} | {{ cor(rolls, 4) }} | {{ cor(rolls, 6) }} |
| `repetition`  | {{ cor(repetition, 1) }} | {{ cor(repetition, 2) }} | {{ cor(repetition, 4) }} | {{ cor(repetition, 6) }} |
| `sorted_rolls`  | {{ cor(sorted_rolls, 1) }} | {{ cor(sorted_rolls, 2) }} | {{ cor(sorted_rolls, 4) }} | {{ cor(sorted_rolls, 6) }} |
| `duplicated`  | {{ cor(duplicated, 1) }} | {{ cor(duplicated, 2) }} | {{ cor(duplicated, 4) }} | {{ cor(duplicated, 6) }} |
| `normal`  | {{ cor(normal, 1) }} | {{ cor(normal, 2) }} | {{ cor(normal, 4) }} | {{ cor(normal, 6) }} |
| `uniform`  | {{ cor(uniform, 1) }} | {{ cor(uniform, 2) }} | {{ cor(uniform, 4) }} | {{ cor(uniform, 6) }} |
